In [1]:
import numpy as np
import random
import time
from IPython.display import clear_output
import gym
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
env = gym.make("FrozenLake-v0")
INPUT_DIM = 1
OUTPUT_DIM = env.action_space.n * env.observation_space.n
print(OUTPUT_DIM)

64


In [3]:
def calculateAvgReward(num_episodes,avg_num_episodes,rewards_all_episodes):
        results = []
        rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/avg_num_episodes)
        count = avg_num_episodes
        for r in rewards_per_thousand_episodes:
#             avg_per_thousand_episode = str(sum(r/self.avg_num_episodes))
            sum_of_rewards = sum(r/avg_num_episodes)
            results.append({"episode":count,"reward":sum_of_rewards})
            count += avg_num_episodes
        return results
    
def createLinePlotForRewards(results,avg_num_episodes):
        df = pd.DataFrame(results)
        fig, ax1 = plt.subplots(figsize=(12, 8))
        sns.lineplot(data=df, x="episode", y="reward", ax=ax1, ci=None)
        ax1.set_ylabel('Rewards',fontsize=13)
        ax1.set_xlabel('Episodes',fontsize=13)
        ax1.set_title("Average reward per "+str(avg_num_episodes)+" episodes",fontsize=16)

In [4]:
class DQNAgent:
    def __init__(self):
        self.num_episodes = 100000
        self.avg_num_episodes = round(self.num_episodes/10)
        self.learning_rate = 0.1
        self.discount_rate = 0.99
        self.exploration_rate = 1
        self.max_exploration_rate = 1
        self.min_exploration_rate = 0.01
        self.exploration_decay_rate = 0.001
        self.model = self.build_model()
        self.opt = tf.optimizers.Adam(0.001)
    
    def build_model(self):
        inputs = tf.keras.layers.Input(INPUT_DIM)
        net = tf.keras.layers.Dense(16,activation='tanh')(inputs)
        net = tf.keras.layers.Dense(16,activation='tanh')(net)
        outputs = tf.keras.layers.Dense(OUTPUT_DIM)(net)
        model = tf.keras.Model(inputs,outputs)
        return model
    def loss_grads(self,state, target):
        with tf.GradientTape(persistent=True) as tape:
            pred = self.model(np.array([state]))
            loss = tf.reduce_mean(tf.square(pred-target))
        grads = tape.gradient(loss,self.model.trainable_variables)
        return loss, grads
    
    def nural_net(self):
    
        rewards_all_episodes = []

        # Q-Learning algorithm
        for episode in range(self.num_episodes):
            state = env.reset()

            done = False
            rewards_current_episode = 0

            while not done: 
                if done == True: 
                    break
                else:
                    # Exploration-exploitation trade-off
                    self.exploration_rate_threshold = random.uniform(0, 1)
                    if self.exploration_rate_threshold > self.exploration_rate:
                        action = np.argmax(self.model.predict([state])) # Exploitation
                        print(action)
                    else:
                        action = env.action_space.sample() # Exploration
                        print('exp act:',action)

                    new_state, reward, done, info = env.step(action)



                    prev_state = state
                    prev_pred = self.model(np.array([prev_state]))
                    prev_action = tf.argmax(prev_pred)
                    
                    curr_state = new_state

                    curr_pred = self.model(np.array([curr_state]))

                    curr_value = tf.reduce_sum(curr_pred)

                    target = prev_pred.numpy()
                    target[prev_action] = reward + self.exploration_decay_rate * curr_value

                    self.exploration_rate *= 0.001

                    loss,grads = self.loss_grads(prev_state, target)
                    self.opt.apply_gradients(zip(grads,self.model.trainable_variables))

                    state = new_state
                    rewards_current_episode += reward
            # Exploration rate decay
            self.exploration_rate = self.min_exploration_rate + \
                (self.max_exploration_rate - self.min_exploration_rate) * np.exp(-self.exploration_decay_rate*episode)


            rewards_all_episodes.append(rewards_current_episode)


        results = calculateAvgReward(self.num_episodes,self.avg_num_episodes,rewards_all_episodes)
        createLinePlotForRewards(results,self.avg_num_episodes)

        return q_table

In [5]:
agent = DQNAgent()
new_qtable = agent.nural_net()

exp act: 2
0
0
0
20


KeyError: 20